# ewx jr

In [1]:
import logging
import socket
import requests
import time
import json
import os
import sys
import re
import flask
import urllib
import flask
import httplib2
import hashlib
import googleapiclient
import oauth2client
import energyworx_client
import energyworx_public
import google.oauth2.credentials

from retrying import retry
from datetime import datetime
from energyworx_client import client
from googleapiclient import discovery
from google_auth_oauthlib import flow
from requests.auth import HTTPBasicAuth
from oauth2client import GOOGLE_TOKEN_URI
from googleapiclient.errors import HttpError
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
from oauth2client.client import GoogleCredentials, OAuth2Credentials

print('imported modules.')

imported modules.


## read secret file from google api

In [2]:
with open('F:\\client_secret_symbolic_bit.apps.googleusercontent.com.json', 'r') as f:
    secret_file = json.load(f)
    secret = secret_file['web']
    
print('read in super-secret google api credentials.')

read in super-secret google api credentials.


## run oauth consent

In [3]:
SCOPES = ['https://www.googleapis.com/auth/userinfo.email', 'openid']
oauth = OAuth2Session(secret['client_id'], redirect_uri=secret['redirect_uris'][0], scope=SCOPES)
authorization_url, state = oauth.authorization_url('https://accounts.google.com/o/oauth2/auth', access_type="offline", prompt="consent")

print('Please go to %s and authorize access.' % authorization_url)

Please go to https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=348739208671-qm0broa5bkddp3t85qibse6l4kqe58fe.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fstevenhurwitt.com%2Fhub%2Foauth_callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+openid&state=OlFEH61UNCBqbEHGGBI0P09GSeHpyA&access_type=offline&prompt=consent and authorize access.


In [4]:
authorization_response = input('Enter the full callback URL')

Enter the full callback URLhttps://stevenhurwitt.com/hub/oauth_callback?state=OlFEH61UNCBqbEHGGBI0P09GSeHpyA&code=4%2F2QGQEL-7MlCNrOZhs0ebgOzox4vMo8uWRj2IeT1js-3r2Xq35O6dvopBffrj3TWZgHlrKeYAQ0usLMD0DLcNJiM&scope=email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+openid&authuser=1&prompt=consent#


## get token

In [5]:
token = oauth.fetch_token('https://accounts.google.com/o/oauth2/token', authorization_response=authorization_response, client_secret=secret['client_secret'])
print('fetched extra super-secret token.')

fetched extra super-secret token.


In [6]:
token_exp = datetime.fromtimestamp(token['expires_at'])
print('Bearer token expires at: {}.'.format(token_exp.strftime('%m/%d/%Y %H:%M:%S')))

Bearer token expires at: 07/28/2020 01:03:23.


## write amended client secret

In [14]:
secret['type'] = 'authorized_user'
secret['refresh_token'] = token['refresh_token']

with open('F:\\super-secret-google-shit.json', 'w') as g:
    json.dump(secret, g)
    
print('wrote new client secret to F:\\super-secret-google-shit.json')

wrote new client secret to F:\super-secret-google-shit.json


## test request

In [15]:
r = oauth.get('https://www.googleapis.com/oauth2/v1/userinfo')
r.json()

{'id': '116108006493438593342',
 'email': 'steven.hurwitt@external.engie.com',
 'verified_email': True,
 'picture': 'https://lh4.googleusercontent.com/-11HvE0chTJs/AAAAAAAAAAI/AAAAAAAAAAA/7qsrZ7S5SPw/photo.jpg'}

## Set credentials for ewx api

In [16]:
os.environ['JPY_USER'] = "True"
os.environ['REFRESH_TOKEN'] = token['refresh_token']
os.environ['CLIENT_ID'] = secret['client_id']
os.environ['CLIENT_SECRET'] = secret['client_secret']
os.environ['TOKEN_URI'] = secret['token_uri']
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'F:\\super-secret-google-shit.json'
print('set necessary os environment variables.')

set necessary os environment variables.


## EWX API credentials

In [17]:
namespace = 'na.engie.com'
test_namespace = 'na.engie_test.com'
default_api_root = 'https://ewx-live.appspot.com'
other = 'cloud.app.energyworx.com/_ah/api'

creds = GoogleCredentials.get_application_default()

#create ewx client
ewx = client.EWX(namespace, default_api_root, creds)

Creating Externalservice client
Discovering services using https://ewx-live.appspot.com/_ah/api/discovery/v1/apis/ewx/v1/rest.
Client created


In [18]:
ewx.get_datasource('NEPOOL_BECO_25917091008')

HttpError: <HttpError 401 when requesting https://ewx-live.appspot.com/_ah/api/ewx/v1/datasource/get/NEPOOL_BECO_25917091008?alt=json returned "Unexpected error: Client ID is not allowed: 348739208671-qm0broa5bkddp3t85qibse6l4kqe58fe.apps.googleusercontent.com">

## try api request

In [20]:
X_API_KEY="626E2AA48CE14D5EA2B4DB8F2052B9FC"

In [21]:
headers = {"Accept": "application/json", "x-namespace" : namespace}
auth = HTTPBasicAuth("apikey", X_API_KEY)
print(headers)

{'Accept': 'application/json', 'x-namespace': 'na.engie.com'}


In [22]:
response = requests.get('https://ewx-live.appspot.com/_ah/api/ewx/v1/datasource/get/NEPOOL_BECO_25917091008', auth = auth, headers = headers)

In [23]:
response.json()

{'error': {'code': 401,
  'errors': [{'domain': 'global',
    'message': 'Unexpected error: Could not retrieve token info with token YXBpa2V5OjYyNkUyQUE0OENFMTRENUVBMkI0REI4RjIwNTJCOUZD',
    'reason': 'required'}],
  'message': 'Unexpected error: Could not retrieve token info with token YXBpa2V5OjYyNkUyQUE0OENFMTRENUVBMkI0REI4RjIwNTJCOUZD'}}